# 6.2 Классификация.Фильмы

Для тех, кто хочет поставить точку в рекомендательных системах.

### Рекомендательная система и kNN

В прошлый раз мы использовали kMeans, тоже метрический алгоритм, но кластеризации, для построения "рекомендательной ситсемы". На самом деле тогда вы строили кластеры жанров/фильмов на основе предпочтений пользователей.  

Теперь попробуем kNN для рекомендаций. Мы построим модель, в которой не будет кластеров, но которая сможет выводить список наиболее похожих фильмов для заданного.

Подготовка датасета похожа на то, что мы уже делали. Отличаться будет только последний пункт, когда мы запустим kNN с косинусной метрикой.

Будем использовать известный нам [датасет с оценками фильмов](http://www.cs.umn.edu/GroupLens). Только в этот раз мы не будем кластеризовывать жанры, а будем рекомендовать похожие фильмы, основываясь на оценках других пользователей. То есть мы будем по факту предсказывать предпочтение нового пользователя на основе других.

Для рекомендательной системы в датасете в объектами будут пользователи, а признаками - оценки фильма данным пользователем.

![](https://i.imgur.com/hn3xjZl.png)

Мы будем использовать метрический алгоритм для поиска ближайших соседей, тем самым рекомендовать фильмы близкие к данному.

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

In [4]:
colab = False 
if colab:
    from google.colab import drive
    drive.mount('/content/drive/')

if colab:
    movies = pd.read_csv('/content/drive/My Drive/Data/imdb_movies.csv.gz', index_col='movieId')
    ratings = pd.read_csv('/content/drive/My Drive/Data/imdb_ratings.csv.gz')
else:
    movies = pd.read_csv('../Data/imdb_movies.csv.gz', index_col='movieId')
    ratings = pd.read_csv('../Data/imdb_ratings.csv.gz')
    
print(movies.shape)
print(ratings.shape)

(9125, 2)
(100004, 4)


In [5]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### 1.1 Поиск популярных фильмов (3 балла)

Среди всех фильмов есть не популярные, для которых выставлена 1-2 оценки. Такого количества данных не достаточно, поэтому нужно срезать много фильмов с маленьким числом отзывов.

Посчтитайте количество оценок для каждого фильма.

In [7]:
movies_stats = pd.DataFrame(data=None, index=movies.index)
movies_stats['ratings_count'] = 0
movies_stats['ratings_count'] = ratings.groupby('movieId').size()
movies_stats.head()

,ratings_count
movieId,
1,247.0
2,107.0
3,59.0
4,13.0
5,56.0


Выведите сводную статистику по количеству оценок фильмов. (общее число фильмов, среднее количество оценок, дисперсию, квантили)

In [8]:
movies_stats.describe()

,ratings_count
count,9066.000000
mean,11.030664
std,24.050800
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,341.000000


Как мы видим `75%` квантиль показывет, что `75%` фильмов имело очень мало оценок. Нам столько будет недостаточно. Посчтитайте квантили в диапазоне от `75%` до `100%` с шагом в `1%`. Выберите тот квантиль, при котором количество оценок равно 20. Напишите какой это квантиль. Напишите сколько всего фильмов останется, если отбросить все фильмы у которых рейтинг меньше этого квантиля.

In [9]:
range_i = np.arange(0.75, 1., 0.01)
n_ratings = 20
quantile = 0
for i in range_i:
    if movies_stats.quantile(i)['ratings_count'] == n_ratings:
        quantile = i
print("quantile =", "%.2f" % quantile)
movies_stats = movies_stats[movies_stats['ratings_count'] >= n_ratings]
print("movies count =", len(movies_stats))

quantile = 0.86
movies count = 1303


### 1.2 Соединяем таблицы (1 балл)

Будет немного не удобно искать фильмы по их идентификаторам, нам нужны названия. Поэтому соедините две таблицы `ratings` и `movies` по `movieId` и отфильтруйте отзывы на редкие фильмы, которые мы нашли в предыдущем шаге. Выведите первые 5 элементов полученного датасета.

In [10]:
movies_ratings = ratings.join(movies, on='movieId')
movies_ratings = movies_ratings[~movies_ratings['movieId'].isin(movies_stats.index)]
movies_ratings.head()

,userId,movieId,rating,timestamp,title,genres
41,2,248,3.0,835355896,Houseguest (1994),Comedy
51,2,314,4.0,835356044,"Secret of Roan Inish, The (1994)",Children|Drama|Fantasy|Mystery
65,2,382,3.0,835356165,Wolf (1994),Drama|Horror|Romance|Thriller
66,2,405,2.0,835356246,Highlander III: The Sorcerer (a.k.a. Highlande...,Action|Fantasy
80,2,537,4.0,835356199,Sirens (1994),Drama


### 1.3 Матрица Фильмы-Пользователи

В нашем случае мы будем искать похожие фильмы, поэтому сделайте из датасета оценок матрицу, где в строках фильмы, в колонках пользователи, а в ячейках оценки. Пропущенные значение, то есть отсутсвие оценки, заполните 0.

Для увеличения скорости выполнения алгоритма ближайших соседей примените функцию [`scipy.sparse.csc_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html),  которая создаст плотное представление нашей разреженой матрицы.

In [11]:
user_movies = pd.DataFrame(index=movies_ratings['title'].unique(), columns=movies_ratings['userId'].unique())
for i in movies_ratings.index:
    user = movies_ratings['userId'][i]
    movie = movies_ratings['title'][i]
    user_movies[user][movie] = movies_ratings['rating'][i]
user_movies = user_movies.fillna(0)
user_movies.head()

,2,3,4,6,7,8,9,10,11,12,...,661,662,663,664,665,667,668,669,670,671
Houseguest (1994),3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
"Secret of Roan Inish, The (1994)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wolf (1994),3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994),2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sirens (1994),4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
matrix = csr_matrix(user_movies)
user_movies = pd.DataFrame(data=matrix.toarray(), index=user_movies.index, columns=user_movies.columns)
user_movies.head()

,2,3,4,6,7,8,9,10,11,12,...,661,662,663,664,665,667,668,669,670,671
Houseguest (1994),3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
"Secret of Roan Inish, The (1994)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wolf (1994),3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994),2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sirens (1994),4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.4 Тренировка модели рекомендаций 

Так как по сути у нас нет каких-то меток классов, мы будем использовать алгоритма без учителя [`sklearn.neighbors.NearestNeighbors`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html). Научите его по данным нашей матрицы пользователей-фильмов и напишите **функцию**, принимающую на вход название фильма и возвращающую список из 6 рекомендованых фильмов, начиная от более похожего.

Для определения "похожести" двух фильмов мы будем использовать косинусную меру, вместо евклидова или минковского:

$$\LARGE cos(\overrightarrow{x},\overrightarrow{y}) = \frac{\overrightarrow{x} \cdot \overrightarrow{y}}{||\overrightarrow{x}|| \times ||\overrightarrow{y}||}$$

In [26]:
neigh = NearestNeighbors(n_neighbors=6, metric='cosine')
neigh.fit(user_movies)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [14]:
matrix.shape

(7762, 596)

In [15]:
user_movies.loc['Wolf (1994)']

2      3.0
3      0.0
4      0.0
6      0.0
7      0.0
      ... 
667    0.0
668    0.0
669    0.0
670    0.0
671    0.0
Name: Wolf (1994), Length: 596, dtype: float64

In [27]:
smth = user_movies.loc['Wolf (1994)']
smth

2      3.0
3      0.0
4      0.0
6      0.0
7      0.0
      ... 
667    0.0
668    0.0
669    0.0
670    0.0
671    0.0
Name: Wolf (1994), Length: 596, dtype: float64

In [17]:
smth.shape

(1, 596)

In [31]:
#something = user_movies.loc['Wolf (1994)'].values
#print(something.reshape(1, len(something)))
#print(something.shape)
indices = neigh.kneighbors(X=[user_movies.loc['Wolf (1994)']], return_distance=False)
indices

array([[   2, 1456, 1455, 1351, 1650, 3028]])

In [22]:
kneighbor

NameError: name 'kneighbor' is not defined

In [ ]:
def similar(name='Pulp Fiction (1994)'):
    # Тут какой-то код
    pass

Выведите рекомендации для фильмов:
- `Pulp Fiction (1994)`
- `Star Wars: Episode V - The Empire Strikes Back (1980)`
- `Lord of the Rings: The Two Towers, The (2002)`

In [ ]:
similar('Pulp Fiction (1994)')

In [ ]:
similar('Star Wars: Episode V - The Empire Strikes Back (1980)')

In [ ]:
similar('Lord of the Rings: The Two Towers, The (2002)')

### 1.5 Как посчитать качество рекомендации?

Как посчитать качество нашей классификации? У нас нет учителя, нет отложенной выборки. Мы просто посчитали все рейтинги и на основе них сделали предположения о похожести товаров. 

А как вы думаете можно проверить качество нашей рекомендательной системы? Напишите развернутый ответ своими словами.

???

![](https://69.media.tumblr.com/06a071083b3e62cd76f31af07ecb895f/tumblr_p49r4rWXG21uxovwqo1_540.gif)